<a href="https://colab.research.google.com/github/TheLargeCheddar/327_project7/blob/main/327Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ensure the current working directory is /content.

In [8]:
!pwd

/content


in case the directory isn't in /content run this.

In [7]:
!cd /content

Make sure the proper libraries are installed, if they are not, use the following pip install commands.

In [ ]:
!pip install numpy

In [ ]:
!pip install pandas

In [ ]:
!pip install scikit-learn

In [14]:
# You will need spotipy library installed, as well as sklearn / pandas/ numpy

import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


the gdown library is required for installing the dataset in google colab envionment.

In [ ]:
# Install the gdown library for downloading the dataset from my google drive
!pip install gdown

This installs the zipped dataset into the google colab enviornment under /content

In [11]:
!gdown 'https://drive.google.com/uc?id=1BJZSaYzkSW1fcrFv7rA_wKhcxChVKpr0'

Downloading...
From: https://drive.google.com/uc?id=1BJZSaYzkSW1fcrFv7rA_wKhcxChVKpr0
To: /content/data.zip
100% 17.3M/17.3M [00:00<00:00, 78.0MB/s]


this unzips the content and makes it available under /content

In [12]:
!unzip data.zip -d /content

Archive:  data.zip
  inflating: /content/data/data.csv  
  inflating: /content/data/data_by_artist.csv  
  inflating: /content/data/data_by_genres.csv  
  inflating: /content/data/data_by_year.csv  
  inflating: /content/data/data_w_genres.csv  


Run the following cells for reading, clustering, and reformatting the data.

In [15]:
#Read data from kaggle data set # this is for google colab
data = pd.read_csv("/content/data/data.csv")
genreData = pd.read_csv('/content/data/data_by_genres.csv')
yearData = pd.read_csv('/content/data/data_by_year.csv')

In [16]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

#K-means clustering algorithm divides the genres in this dataset into ten clusters
#based on audio features of each genre
cluster_pipeline = Pipeline([('scaler', StandardScaler()), ('kmeans', KMeans(n_clusters=10))])
X = genreData.select_dtypes(np.number)
cluster_pipeline.fit(X)
genreData['cluster'] = cluster_pipeline.predict(X)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [17]:
#K-means then divides the songs into 20 clusters
#based on audio features of each song
song_cluster_pipeline = Pipeline([('scaler', StandardScaler()),
                                  ('kmeans', KMeans(n_clusters=20,
                                   verbose=False,))
                                 ], verbose=False)

X = data.select_dtypes(np.number)
numberCols = list(X.columns)
song_cluster_pipeline.fit(X)
song_cluster_labels = song_cluster_pipeline.predict(X)
data['cluster_label'] = song_cluster_labels

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


You may need to install spotipy, if so run this pip install command

In [19]:
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.0/252.0 kB 9.4 MB/s eta 0:00:00


In [20]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from collections import defaultdict

#connection to spotify api
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="6829ed3b61a14b32bc343317e1978458",
                                                           client_secret="3029e57d496d4830afcc09d56be5bbc3"))

#Searches database by song and year for song, returns song data
def findSong(name, year):
    songData = defaultdict()
    results = sp.search(q= 'track: {} year: {}'.format(name,year), limit=1)
    if results['tracks']['items'] == []:
        return None

    results = results['tracks']['items'][0]
    track_id = results['id']
    audio_features = sp.audio_features(track_id)[0]

    songData['name'] = [name]
    songData['year'] = [year]
    songData['explicit'] = [int(results['explicit'])]
    songData['duration_ms'] = [results['duration_ms']]
    songData['popularity'] = [results['popularity']]

    for key, value in audio_features.items():
        songData[key] = value

    return pd.DataFrame(songData)

In [21]:
from collections import defaultdict
from scipy.spatial.distance import cdist
import difflib

#song attributes
numberCols = ['valence', 'year', 'acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'popularity', 'speechiness', 'tempo']


#Uses function findSong to locate and return data of song
def getSongData(song, spotifyData):
    try:
        songData = spotifyData[(spotifyData['name'] == song['name']) & (spotifyData['year'] == song['year'])].iloc[0]
        return songData
    except IndexError:
        songData = findSong(song['name'], song['year'])
        if songData is not None:
            songData = songData.iloc[0]
        return songData


#Calculates the mean (average) of the matrix along the vertical axis (axis=0)
#resulting in a single vector that represents the average features of all included songs.
def getMeanVector(songList, spotifyData):
    songVectors = []
    for song in songList:
        songData = getSongData(song, spotifyData)
        if songData is None:
            print('Warning: {} does not exist in Spotify or in database'.format(song['name']))
            continue
        songVector = songData[numberCols].values
        if songVector.shape[0] == len(numberCols):  # Check if song_vector has the correct length
            songVectors.append(songVector)
        else:
            print('Warning: Incomplete data for {}'.format(song['name']))
            continue

    if not songVectors:
        return None
    songMatrix = np.array(songVectors)
    return np.mean(songMatrix, axis=0)

#reformats song dict list for use in reccomendSongs function
def flattenDict(dict):

    flattenedDict = defaultdict()
    for key in dict[0].keys():
        flattenedDict[key] = []

    for dictionary in dict:
        for key, value in dictionary.items():
            flattenedDict[key].append(value)

    return flattenedDict

#requires song list, spotify data, and will determine the songs to reccomend. Returns 10 reccomended songs.
def recommendSongs( songList, spotifyData, numSongs=10):
    #print(spotify_data.head())  # Check the first few rows of the data
    metadataCols = ['name', 'year', 'artists']
    songDict = flattenDict(songList)

    songCenter = getMeanVector(songList, spotifyData)
    scaler = song_cluster_pipeline.steps[0][1]
    scaledData = scaler.transform(spotifyData[numberCols])
    scaledSongCenter = scaler.transform(songCenter.reshape(1, -1))
    distances = cdist(scaledSongCenter, scaledData, 'cosine')
    index = list(np.argsort(distances)[:, :numSongs][0])

    #resSongs will be the resulting list of reccomended songs
    resSongs = spotifyData.iloc[index]
    resSongs = resSongs[~resSongs['name'].isin(songDict['name'])]
    return resSongs[metadataCols].to_dict(orient='records')


Feel free to input your own songs into the function as long as it follows the format. The dataset is large, but If the song isn't in the dataset it won't work, you will need to change the song and rerun. the size of the input songList can vary.

In [26]:
#make sure song list format is "name: **, year: **"
recommendSongs([{'name': 'Them Changes', 'year':2017},
                {'name': 'WEIGHT OFF', 'year': 2016},
                {'name': 'Ordinary Pleasure', 'year': 2019},
                {'name': 'Deep Down Body Thurst', 'year': 2017},
                {'name': 'Some', 'year': 2017}],  data)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


[{'name': "survivin'", 'year': 2020, 'artists': "['Bastille']"},
 {'name': 'Dembow 2020',
  'year': 2020,
  'artists': "['Yandel', 'Rauw Alejandro']"},
 {'name': 'COMO UN BEBÉ',
  'year': 2019,
  'artists': "['J Balvin', 'Bad Bunny', 'Mr Eazi']"},
 {'name': 'Tonight Tonight', 'year': 2011, 'artists': "['Hot Chelle Rae']"},
 {'name': 'Singapur',
  'year': 2020,
  'artists': "['El Alfa', 'Chael Produciendo']"},
 {'name': 'Good Girl', 'year': 2020, 'artists': "['Dustin Lynch']"},
 {'name': 'New Man', 'year': 2017, 'artists': "['Ed Sheeran']"},
 {'name': 'Djadja (feat. Maluma) - Remix',
  'year': 2020,
  'artists': "['Aya Nakamura', 'Maluma']"},
 {'name': 'Créeme', 'year': 2018, 'artists': "['KAROL G', 'Maluma']"},
 {'name': 'Lonely If You Are', 'year': 2020, 'artists': "['Chase Rice']"}]